In [115]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

In [116]:
games_2021 = pd.read_json("data/games_2021.json")
big_10_teams = pd.read_json('data/big_10_teams_2021.json')

df = pd.merge(big_10_teams, games_2021)

df

,id,teams,season,week,season_type,start_date,start_time_tbd,neutral_site,conference_game,attendance,...,away_conference,away_division,away_points,away_line_scores,away_post_win_prob,away_pregame_elo,away_postgame_elo,excitement_index,highlights,notes
0,401331447,"[{'school': 'Michigan', 'conference': 'Big Ten...",2021,14,regular,2021-12-05T01:00:00.000Z,False,True,True,67183.0,...,Big Ten,fbs,42,"[14, 0, 7, 21]",0.935983,1988.0,2065.0,1.873741,NaN,None
1,401282179,"[{'school': 'UT San Antonio', 'conference': 'C...",2021,1,regular,2021-09-04T23:30:00.000Z,False,False,False,33906.0,...,Conference USA,fbs,37,"[7, 13, 3, 14]",0.922748,1445.0,1456.0,6.813355,NaN,None
2,401282772,"[{'school': 'Western Michigan', 'conference': ...",2021,1,regular,2021-09-04T16:00:00.000Z,False,False,False,109295.0,...,Mid-American,fbs,14,"[7, 0, 0, 7]",0.001313,1510.0,1429.0,2.323598,NaN,None
3,401282809,"[{'school': 'Ohio State', 'conference': 'Big T...",2021,2,regular,2021-09-11T16:00:00.000Z,False,False,False,100482.0,...,Pac-12,fbs,35,"[0, 14, 14, 7]",0.788766,1700.0,1730.0,7.298784,NaN,None
4,401282812,"[{'school': 'Ohio State', 'conference': 'Big T...",2021,11,regular,2021-11-13T20:30:00.000Z,False,False,True,101009.0,...,Big Ten,fbs,31,"[7, 10, 14, 0]",0.004840,1640.0,1619.0,1.680140,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,401282820,"[{'school': 'Connecticut', 'conference': 'FBS ...",2021,2,regular,2021-09-11T19:00:00.000Z,False,False,False,14187.0,...,Big Ten,fbs,49,"[7, 28, 14, 0]",0.990722,1483.0,1583.0,0.314976,NaN,None
102,401282629,"[{'school': 'Miami', 'conference': 'ACC', 'hom...",2021,3,regular,2021-09-18T16:00:00.000Z,False,False,False,46427.0,...,Big Ten,fbs,38,"[0, 10, 7, 21]",0.906809,1489.0,1554.0,7.347641,NaN,None
103,401282628,"[{'school': 'Duke', 'conference': 'ACC', 'home...",2021,3,regular,2021-09-18T20:00:00.000Z,False,False,False,12323.0,...,Big Ten,fbs,23,"[0, 7, 13, 3]",0.141792,1574.0,1547.0,5.821158,NaN,None
104,401282765,"[{'school': 'Maryland', 'conference': 'Big Ten...",2021,4,regular,2021-09-25T19:30:00.000Z,False,False,False,30117.0,...,Mid-American,fbs,16,"[6, 3, 0, 7]",0.009264,1386.0,1337.0,3.101408,NaN,None


In [117]:
for index, row in df.iterrows():
    for team in row['teams']:
        
        # find home and away teams        
        if team['homeAway'] == 'home':
            # add home teams
            df.loc[index, 'home_team'] = team['school']
            for i in team['stats']:
                # add home stats
                stats = ('home_' + i['category'], i['stat'])
                df.loc[index, stats[0]] = stats[1]

        if team['homeAway'] == 'away':
            # add away teams
            df.loc[index, 'away_team'] = team['school']
            for i in team['stats']:
                # add away stats
                stats = ('away_' + i['category'], i['stat'])
                df.loc[index, stats[0]] = stats[1]

# Move home and away team columns to begininng
first_column = df.pop('home_team')
df.insert(1, 'home_team', first_column)

second_column = df.pop('away_team')
df.insert(2, 'away_team', second_column)

# Drop 'teams' columns
df = df.drop('teams', axis=1)

# First row of df
df.iloc[0].dropna()

id                      401331447
home_team                    Iowa
away_team                Michigan
season                       2021
week                           14
                          ...    
home_netPassingYards          175
home_totalYards               279
home_fourthDownEff            0-1
home_thirdDownEff            5-19
home_firstDowns                15
Name: 0, Length: 93, dtype: object

In [118]:
df = df[df["home_points"].notna()]
df = df[df["away_points"].notna()]

df = df[df["home_rushingTDs"].notna()]
df = df[df["home_turnovers"].notna()]
df = df[df["home_yardsPerRushAttempt"].notna()]

# Makes a new column called "home_win" with value True
# if home_points is greater than away_points, False otherwise 
df["home_win"] = df["home_points"] > df["away_points"]

In [119]:
# Here your features (X) will just be rows with game attendance.
# Your labels (y) will be whether home team won that game or not
X = df[["home_rushingTDs", "home_turnovers", "home_yardsPerRushAttempt"]]
y = df["home_win"]


# 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
print(len(X_train))
print(len(X_test))

79
27


In [120]:
clf = SVC(kernel='linear', C = 1.0)
clf.fit(X_train, y_train)

# score_train = clf.score(X_train, y_train)
score_test = clf.score(X_test, y_test)

print(score_test)

0.8518518518518519


79
27


0.7407407407407407


79
27


0.8518518518518519
